# K-MCI
### Intro
In this notebook we have the main algorithm of the article. All necessary functions are implemented in function.py file.<br>
This document have blow sections:<br>
1. Importing Libraries & classes
2. Reading datasets
3. Preprocessing datasets
4. Main Algorithm

### Importing Libraries & classes
We import some libraries from third party.

In [82]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import random
import copy
from sklearn import preprocessing

In [2]:
# defnition of Candidate class
class Candidate:    
    def __init__(self,nodes,sampling_interval,fitness=-1):
        self.nodes = nodes
        self.sampling_interval = sampling_interval 
        self.variation = []
        self.random_number
        self.fitness = fitness
        self.centers = []
        self.probability = []
        self.labels = []
        self.each_fitness = []
        
    def random_number(self):
        self.random_number = random.random()
        
    def describe(self):
        print("Candidate with #{} nodes, #{} centers, fitness={}\n clusters:\n{} \n Centers:\n{}\n sampling_interval:\n{}"
              .format(len(self.nodes),len(self.centers),self.fitness,self.nodes[np.random.randint(self.nodes.shape[0], size=5), :],self.centers,self.sampling_interval))

### Reading Datasets and Preprocessing
We show the structure of our data for all of datasets.<br>
We have these datasets from UCI Machine Learning Repository:
1. Breast Cancer Wisconsin
2. Contraceptive Method Choice data
3. Glass
4. Iris
5. Vowel
6. Wine

In [114]:
""" In this cell we have some code which will do preprocessing for us on all datasets """
""" the resourses of these datasets are mentioned in README.md file. """

path = 'datasets/' 
# because we are using local files, you need to download datasets and change the "path" variable
# local folder of your downloaded datasets


def bcw(): 
    # importing dataset
    dataset = pd.read_csv(path+'bcw.csv',
                          names = ['Sample code number','Clump Thickness','Uniformity of Cell Size',
                                   'Uniformity of Cell Shape','Marginal Adhesion','Single Epithelial',
                                   'Bare Nuclei','Bland Chromatin','Normal Nucleoli','Mitoses','Class'])
    dataset = dataset.drop(dataset.columns[0], axis=1)
    dataset= dataset.replace(to_replace='?',value=np.nan)
    dataset = dataset.dropna(axis =0) # resolving missing values
    dataset = dataset.astype('int64') # casting string valued column to int64
    x = dataset.iloc[:,:-1].values # features
    y = dataset.iloc[:,-1].values # target values
    return (x,y,dataset)


def cmc():
    # importing dataset
    dataset = pd.read_csv(path+'cmc.csv', names=["Wife's age","Wife's education","Husband's education",
                                                 "Number of children ever born","Wife's religion",
                                                 "Wife's now working?","Husband's occupation",
                                                 "Standard-of-living index","Media exposure","Contraceptive method used"])
    x = dataset.iloc[:,:-1].values # features
    y = dataset.iloc[:,-1].values # target values
    return (x,y,dataset)


def glass():
    # importing dataset
    dataset = pd.read_csv(path+'glass.csv', names= ['Id','refractive index','Sodium','Magnesium','Aluminum','Silicon','Potassium'
                                                    ,'Calcium','Barium','Iron','glass'])
    x = dataset.iloc[:,1:10].values # features
    y = dataset.iloc[:,-1].values # target values
    return (x,y,dataset)


def iris():
    # importing dataset
    dataset = pd.read_csv(path+'iris.csv', names= ['sepal length','sepal width','petal length','petal width','class'])
    x = dataset.iloc[:,:-1].values # features
    y = dataset.iloc[:,-1].values # target values
    
    # encoding categorial data types to labelEncoder
    from sklearn.preprocessing import LabelEncoder
    labelencoder_y = LabelEncoder()
    labelencoder_y = labelencoder_y.fit(y)
    y = labelencoder_y.transform(y)  # 0 for 'Iris-setosa', 1 for 'Iris-versicolor', 2 for 'Iris-virginica'
    return (x,y,dataset)


def vowel():
    # importing dataset
    dataset = pd.read_csv(path+'vowel.csv', names= ['vowel','type 1 frq','type 2 frq','type 3 frq'])
    x = dataset.iloc[:,1:].values # features
    y = dataset.iloc[:,0].values # target values
    return (x,y,dataset)

def wine():
    # importing dataset
    dataset = pd.read_csv(path+'wine.csv', names= ['class','Alcohol','Malic acid','Ash','Alcalinity of ash','Magnesium','Total phenols','Flavanoids','Nonflavanoid phenols','Proanthocyanins','Color intensity','Hue','OD280/OD315','Proline'])
    x = dataset.iloc[:,1:].values # features
    y = dataset.iloc[:,0].values # target values
    return (x,y,dataset)

In [115]:
# Getting data from our preprocessing class Datasets
x_bcw, y_bcw, df_bcw = bcw()
x_cmc, y_cmc, df_cmc = cmc()
x_glass, y_glass, df_glass = glass()
x_iris, y_iris, df_iris = iris()
x_vowel, y_vowel, df_vowel = vowel()
x_wine, y_wine, df_wine = wine()

In [62]:
# Showing dataset of Breast Cancer Wisconsin
df_bcw.head(3)

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,5,1,1,1,2,1,3,1,1,2
1,5,4,4,5,7,10,3,2,1,2
2,3,1,1,1,2,2,3,1,1,2


In [120]:
# Showing dataset of Contraceptive Method Choice data
df_cmc.head(3)

,Wife's age,Wife's education,Husband's education,Number of children ever born,Wife's religion,Wife's now working?,Husband's occupation,Standard-of-living index,Media exposure,Contraceptive method used
0,24,2,3,3,1,1,2,3,0,1
1,45,1,3,10,1,1,3,4,0,1
2,43,2,3,7,1,1,3,4,0,1


In [121]:
# Showing dataset of Glass
df_glass.head(3)

,Id,refractive index,Sodium,Magnesium,Aluminum,Silicon,Potassium,Calcium,Barium,Iron,glass
0,1,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,2,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,3,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1


In [122]:
# Showing dataset of Iris
df_iris.head(3)

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa


In [123]:
# Showing dataset of Vowel
df_vowel.head(3)

,vowel,type 1 frq,type 2 frq,type 3 frq
0,1,700,1500,2600
1,1,550,1550,2400
2,1,700,1500,2600


In [71]:
# Showing dataset of Wine
df_wine.head(3)

,class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185


### Standardization
In this cell, we standard our data using this equation: <img src='http://3.bp.blogspot.com/_xqXlcaQiGRk/RpO4yR0oKtI/AAAAAAAAABM/7rUWCMwizus/s200/fig2.png'></img>

In [128]:
x_bcw = preprocessing.scale(x_bcw)
x_wine = preprocessing.scale(x_wine)
x_cmc = preprocessing.scale(x_cmc)
x_glass = preprocessing.scale(x_glass)
x_iris = preprocessing.scale(x_iris)
x_vowel = preprocessing.scale(x_vowel)

C:\Program Files\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


# Main Algorithm 

The implementation of main algorithm have below sections:
1. <Strong>Initializing Parameters and Candidates</Strong>
    1. Sampling Interval
    2. Apply Random Centers to Candidates
    3. Visualizing Candidates
2. <Strong>Running Kmean to Exploit</Strong>
    1. K-means Algorithm
    2. Visualizing Candidates
3. <Strong>Mutation Algorithm</Strong>
    1. Fitness over Mutated Candidate
    2. Probablility Equation
    3. Mutation Logic

**Extra details will be explain in other next sections**


## Initializing Parameters and Candidates
In this step, we initialize our algorithm. There are some parameters which are important to convergance speed and quality of solution. So we initialize them here based on the paper.<br>
Each time you want to run K-MCI algorithm, you should first do this initialization.<br>

In [117]:
# notice: this values related to IRIS dataset
candidates_array = [] # the array of all candidates
number_of_candidates = 5
sampling_interval_reduction_factor = 0.95
convergence_parameter = None # what is this in paper???
mutation_random = 0.7
iterations_count = 3500
variations_count = 15
number_of_clusters = 3 # in this article number of clusters are predefined

## Sampling Interval
this function use for making sampling_interval


In [118]:
def sampling_intervals(input_array):
    Maxes = np.amax(input_array,axis=0)
    Mins  = np.amin(input_array,axis=0)
    return (Maxes,Mins)

In [119]:
# initializing candidates
for i in range(0 ,number_of_candidates):
    candidates_array.append(Candidate(x_wine,sampling_interval = sampling_intervals(x_wine)))

## Visualizing Candidates
some parameters are uninitialized. They will have value after running Kmean and Cohort and mutations.

In [120]:
candidates_array[random.randint(0,number_of_candidates-1)].describe()

Candidate with #178 nodes, #0 centers, fitness=-1
 clusters:
[[ 0.38947368  0.19565217  0.3315508   0.51030928  0.16304348  0.42068966
   0.33333333  0.35849057  0.33753943  0.14163823  0.45528455  0.84249084
   0.2810271 ]
 [ 0.          0.15217391  0.44919786  0.56185567  0.16304348  0.51034483
   0.38607595  0.73584906  0.50473186  0.05290102  1.          0.58608059
   0.09201141]
 [ 0.30789474  0.45256917  0.51336898  0.43298969  0.2826087   0.09310345
   0.03164557  0.50943396  0.10094637  0.36006826  0.14634146  0.20512821
   0.16547789]
 [ 0.46842105  0.31027668  0.55614973  0.69072165  0.30434783  0.05862069
   0.15822785  0.26415094  0.13249211  0.37713311  0.14634146  0.03296703
   0.20114123]
 [ 0.63684211  0.58498024  0.6631016   0.63917526  0.44565217  0.24827586
   0.12236287  0.56603774  0.33123028  0.80204778  0.30081301  0.10622711
   0.29743224]] 
 Centers:
[]
 sampling_interval:
(array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.]), array([ 0.,  0

## Apply Random Centers to Candidates
In this step, we just select some random nodes as centers with psuedorandom functions in python native library.

In [121]:
for c in range(0,number_of_candidates):
    candidates_array[c].centers =  np.asarray(random.sample(list(candidates_array[c].nodes),number_of_clusters))
    print(candidates_array[c].centers)

[[ 0.66578947  0.1916996   0.50802139  0.28865979  0.51086957  0.74827586
   0.62236287  0.39622642  0.60883281  0.41382253  0.38211382  0.77289377
   0.36875892]
 [ 0.20789474  0.19367589  0.27807487  0.45876289  0.17391304  0.52413793
   0.2742616   0.45283019  0.31861199  0.0665529   0.37398374  0.42857143
   0.09771755]
 [ 0.56315789  0.87944664  0.51336898  0.58762887  0.25        0.26206897
   0.06118143  0.90566038  0.35962145  0.56484642  0.09756098  0.07692308
   0.31883024]]
[[ 0.7         0.49802372  0.63101604  0.48453608  0.40217391  0.29310345
   0.0464135   0.69811321  0.12302839  0.39249147  0.3902439   0.2014652
   0.28673324]
 [ 0.21315789  0.02964427  0.65240642  0.3814433   0.26086957  0.42068966
   0.39451477  0.16981132  0.61198738  0.15102389  0.25203252  0.66300366
   0.17261056]
 [ 0.1         0.          0.60962567  0.53608247  0.19565217  0.51724138
   0.35232068  0.54716981  0.32492114  0.15358362  0.50406504  0.38095238
   0.11126961]]
[[ 0.          0.1521

## K-means Algorithm 
this function fitting k-means model on dataset

In [122]:
def doKmeans(x,clusters_count,init_centers):
    kmeans = KMeans(n_clusters = clusters_count, init = init_centers,n_init = 50)
    kmeans = kmeans.fit(x)
    labels = kmeans.labels_
    return (labels,kmeans.cluster_centers_,kmeans.inertia_)


## Running Kmean to Exploit
Here by running Kmean on all of our candidates, they centers initialized and nodes assigns to clusters based on Kmean++ algorithm.<br>
The objective function of Kmean is also the object function of K-MCI. <br>
So the fitness of any candidates is calculated using Kmean inertia or mean square error of nodes to their centers distances.

In [125]:

for c in range(0,number_of_candidates):
    y_predict_temp,centers_temp,fitness_temp = doKmeans(candidates_array[c].nodes,clusters_count = number_of_clusters,init_centers = candidates_array[c].centers)
    candidates_array[c].centers = centers_temp
    candidates_array[c].fitness = fitness_temp
    candidates_array[c].labels   = y_predict_temp
    
candidates_array[2].labels

C:\Program Files\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:896: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=50
  return_n_iter=True)


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

## Visualizing Candidates
In this step, by Kmeans, we assigned centers to each candidates and calculated the fitness of each candidate using inertia

In [126]:
candidates_array[random.randint(0,number_of_candidates-1)].describe()

Candidate with #178 nodes, #3 centers, fitness=48.985414960044636
 clusters:
[[ 0.53157895  0.1798419   0.63636364  0.3814433   0.30434783  0.50689655
   0.44092827  0.30188679  0.32492114  0.25341297  0.5203252   0.45421245
   0.58987161]
 [ 0.          0.15217391  0.44919786  0.56185567  0.16304348  0.51034483
   0.38607595  0.73584906  0.50473186  0.05290102  1.          0.58608059
   0.09201141]
 [ 0.78684211  0.18577075  0.45454545  0.27835052  0.2826087   0.57586207
   0.41983122  0.24528302  0.49526814  0.29180887  0.45528455  0.84981685
   0.53994294]
 [ 0.54736842  0.05335968  0.18181818  0.22680412  0.08695652  0.68965517
   0.59915612  0.24528302  0.58990536  0.34300341  0.5203252   0.6996337
   0.15977175]
 [ 0.72105263  0.22924901  0.70588235  0.33505155  0.48913043  0.69655172
   0.51687764  0.49056604  0.40063091  0.42832765  0.52845528  0.60805861
   0.78245364]] 
 Centers:
[[ 0.55371517  0.50736263  0.56558666  0.54851425  0.3115942   0.24273158
   0.10101762  0.607473

In [127]:
sum_of_squares(candidates_array[0].nodes,candidates_array[0].centers,candidates_array[0].labels)

48.985414960044636

## Fitness over Mutated Candidate
Just simple sum of square of distances

In [25]:
def sum_of_squares(data, centroids, labels):
    sqe = 0
    for l in np.unique(labels):
        data_l = data[labels == l]
        resid = data_l - centroids[l]
        sqe += (resid**2).sum()
    return sqe

### Mean Square Distance to a Specific Center
This function is exactly based on "sum_of_squares" function. But here all centers not engaged because we want to find the sum of square distances of nodes of a cluster to their cluster center.<br>
The idea is when mutation process done, at least one dimension may get better center but the other goes bad. So we want to have that center which is better to the previous time.

In [134]:
def center_dist(data, center, labels,center_position):
    sqe = 0
    for l in np.unique(labels):
        data_l = data[labels == l][center_position]
        resid = data_l - center
        sqe += (resid**2).sum()
    return sqe

In [135]:
def each_fitness(candidates_array_example):
    fitness_of_each_centers = [[0 for x in range(len(candidates_array_example.centers[0]))]for y in range(len(candidates_array_example.centers))]
    for i in range(0,len(candidates_array_example.centers)):
        for j in range(0,len(candidates_array_example.centers[i])):
            fitness_of_each_centers[i][j] = center_dist(candidates_array_example.nodes,candidates_array_example.centers[i][j],candidates_array_example.labels,j)
    return np.array(fitness_of_each_centers)
print(each_fitness(candidates_array[0]))
print(candidates_array[0].centers)
print(len(candidates_array[0].centers))
print(len(candidates_array[0].centers[0]))


[[ 86.55033058  44.46710744  69.77983471 105.29      ]
 [108.31046875  50.31171875  80.3275     109.84761719]
 [182.39755208  44.48354167  90.78063802  49.59199219]]
[[4.73181818 2.92727273 1.77272727 0.35      ]
 [5.19375    3.63125    1.475      0.271875  ]
 [6.31458333 2.89583333 4.97395833 1.703125  ]]
3
4


## Mutation algorithm 
this function do mutation algorithm on candidates interval's centers

In [136]:
def mutation(candidate_array,mutation_random):
    New_candidate = copy.deepcopy(candidate_array)
    Mutant_candidate = copy.deepcopy(candidate_array)
    Trial_candidate = copy.deepcopy(candidate_array)
    for x in range(len(candidate_array)):
        a = np.full(len(candidate_array), 1/(len(candidate_array)-1))
        a[x] = 0
        temp = np.random.choice(len(candidate_array), 3, replace = False, p=a)
        for i in range(len(candidate_array[0].centers)):
            for j in range(len(candidate_array[0].centers[i])):
                Mutant_candidate[x].centers[i][j] = candidate_array[temp[0]].centers[i][j]+ random.random()*(candidate_array[temp[1]].centers[i][j] - candidate_array[temp[2]].centers[i][j])
                if random.random() < mutation_random:
                    Trial_candidate[x].centers[i][j] = Mutant_candidate[x].centers[i][j] #Trial & Mutant must be copy of same candidate
                    
                    
        Trial_candidate[x].fitness = sum_of_squares(Trial_candidate[x].nodes,Trial_candidate[x].centers,Trial_candidate[x].labels)
        
        if Trial_candidate[x].fitness < sum_of_squares(candidate_array[x].nodes,candidates_array[2].centers,candidates_array[2].labels):
            New_candidate[x] = Trial_candidate[x]
            print("sakjfnr")
        else:
            New_candidate[x] = candidate_array[x]
        #print(Trial_candidate[x].centers)
        #print(candidate_array[x].centers)
        #print(New_candidate[x].centers)
        #print(Mutant_candidate[x].centers)
        print("###########################################################################################################")
        Trial_candidate[x].each_fitness = each_fitness(Trial_candidate[x])
        print(Trial_candidate[x].each_fitness)
        candidate_array[x].each_fitness = each_fitness(candidate_array[x])
        print(candidate_array[x].each_fitness)
        print("###########################################################################################################")
        
    return New_candidate
#print(candidates_array[1].fitness)
#for i in range(0,3500):
New_candidates_array = mutation(candidates_array,mutation_random)
    #if New_candidates_array[1].fitness != candidates_array[1].fitness:
     #   print(New_candidates_array[1].fitness)#never happen

###########################################################################################################
[[102.71650472  44.89391602  69.77983471  92.45118411]
 [108.31046875  44.96151323  88.32152363 126.94927922]
 [237.62779012  44.68889544 533.43214384  49.59199219]]
[[ 86.55033058  44.46710744  69.77983471 105.29      ]
 [108.31046875  50.31171875  80.3275     109.84761719]
 [182.39755208  44.48354167  90.78063802  49.59199219]]
###########################################################################################################
###########################################################################################################
[[253.28040557  47.72498565  94.520528    77.05997213]
 [117.25224133  45.1080466   58.83535071 155.32044413]
 [139.47155716  46.14345657  48.99862305 126.95392889]]
[[ 76.465232    45.350208    94.520528   168.320592  ]
 [117.25224133  48.07917227  58.83535071  92.93291051]
 [183.85863905  45.32946746 110.37976331  67.07786982]]
############

# Probability Equation

In [137]:
def probability(candidates):
    for c in range(0,len(candidates)):
        temp_array = []
        for clusters in range(0,len(candidates[c].centers)):
            temp_array.append((1/np.sum(candidates[c].each_fitness[clusters]))/(np.sum(1/(np.sum(x.each_fitness[clusters])) for x in candidates)))
        print(temp_array)    
        candidates[c].probability = temp_array
    return

### Roulette Wheel Selection
This is a logic for selecting targets with higher probability.<br>
In this article, each candidate want to follow the other candidate with higher fitness. So in the previous section <strong><em>Probability Equation<strong><em>, we calculated all probabilities of candidates based on their fitnesses.

In [138]:
def roulette_wheel_selection(inertia_array):
    maximum = np.sum(inertia_array)
    pick = random.uniform(0, maximum)
    current = 0
    for fitness in inertia_array:
        current += fitness
        if current > pick:
            return fitness

In [139]:
def shrinked_sampling_interval(candidates_array):
    for i in range(0,len(candidates_array)):
        shrinked_sampling_interval = []
        for j in range(0,len(candidates_array[i].centers)):
            Second_layer = []
            for k in range(0,len(candidates_array[i].centers[j])):
                Third_layer = []
                Temp = abs(candidates_array[i].sampling_interval[0][k] - candidates_array[i].sampling_interval[1][k]) * sampling_interval_reduction_factor
                Third_layer.append(candidates_array[i].centers[j][k] - (Temp/2))
                Third_layer.append(candidates_array[i].centers[j][k] + (Temp/2))
                Second_layer.append(Third_layer)
            shrinked_sampling_interval.append(Second_layer)
        candidates_array[i].sampling_interval = shrinked_sampling_interval
    return
shrinked_sampling_interval(New_candidates_array)
print(New_candidates_array[0].sampling_interval)

[[[3.0218181818181824, 6.441818181818182], [1.7872727272727271, 4.067272727272727], [-1.0297727272727277, 4.575227272727273], [-0.7900000000000003, 1.4899999999999995]], [[3.4837499999999997, 6.90375], [2.4912499999999995, 4.77125], [-1.3275000000000001, 4.2775], [-0.8681249999999996, 1.4118750000000002]], [[4.604583333333333, 8.024583333333334], [1.7558333333333334, 4.035833333333334], [2.171458333333333, 7.776458333333333], [0.5631250000000001, 2.8431249999999997]]]


In [149]:
def variation(candidates_array):
    for candidate in candidates_array:
        variations = []
        for i in range(0,variations_count):
            Second_layer = []
            for j in candidate.sampling_interval:
                Third_layer = []
                for k in range(0,len(j)):
                    #print(k)
                    Third_layer.append(random.uniform(j[k][0],j[k][1]))
                Second_layer.append(Third_layer)
            variations.append(Second_layer)
        candidate.variation = variations    
variation(New_candidates_array)
print(New_candidates_array[3].variation[1])

[[7.053495149064516, 3.3599382410025296, 3.6057476175630803, 2.5402954393727946], [4.532479840256706, 3.1840474891336856, -1.1540168609894432, 0.8671900217055385], [5.868836676719992, 3.167380743085452, 8.1906902624868, 1.2681387578290744]]
